### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle
import os
# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
#from Env import CabDriver


Using TensorFlow backend.


In [2]:
# Import routines

import numpy as np
import math
import random
from itertools import permutations,product

# Defining hyperparameters
m = 5 # number of cities, ranges from 1 ..... m
t = 24 # number of hours, ranges from 0 .... t-1
d = 7  # number of days, ranges from 0 ... d-1
C = 5 # Per hour fuel and other costs
R = 9 # per hour revenue from a passenger


class CabDriver():

    def __init__(self):
        """initialise your state and define your action space and state space"""
        action_list = list(permutations(range(0,m) ,2))
        action_list.append((0,0))
        self.action_space = np.array(action_list) #action space is unique 2 values(source & destination) + the no op
        self.state_space = list(product(*[list(range(0,m)), list(range(0,t)), list(range(0,d))])) #State space from MDP:
        #𝑠=𝑋𝑖𝑇𝑗𝐷𝑘 𝑤ℎ𝑒𝑟𝑒 𝑖=0…𝑚−1;𝑗=0….𝑡−1;𝑘=0…..𝑑−1, Where 𝑋𝑖 represents a driver’s current location, 𝑇𝑗 represents time component (more specifically hour of the day), 𝐷𝑘 represents the day of the week
        self.state_size = len(self.state_space)
        self.action_size = len(self.action_space)
        self.state_init = random.choice(self.state_space) #Initialises to any random self_space
        self.encode_vector = np.array([24*7, 7, 1]).reshape(3, 1)


        # Start the first round
        self.reset()


    ## Encoding state (or state-action) for NN input

    def state_encod_arch1(self, curr_state, batch_size=1):
        """convert the state into a vector so that it can be fed to the NN. This method converts a given state into a vector format. Hint: The vector is of size m + t + d."""
       
        #Encoded values of m + t + d
        
        curr_state = np.array(curr_state).reshape(1, 3)
        #print(curr_state.shape)
        #enc_mat = self.encode_vector
        # pos = (state[0]*24*7) + (state[1]*7) + state[2]
        
        pos_mat = np.dot(curr_state, self.encode_vector)
        state_encod =  np.zeros((1, self.state_size))
        # state_encod[pos] = 1
        for i in range(batch_size):
            state_encod[i][pos_mat[i]] = 1

        return np.reshape(state_encod, [1, env.state_size])
    


    # Use this function if you are using architecture-2 
    # def state_encod_arch2(self, state, action):
    #     """convert the (state-action) into a vector so that it can be fed to the NN. This method converts a given state-action pair into a vector format. Hint: The vector is of size m + t + d + m + m."""

        
    #     return state_encod


    ## Getting number of requests

    def requests(self, state):
        """Determining the number of requests basis the location. 
        Use the table specified in the MDP and complete for rest of the locations"""
        location = state[0]
        requests = 0
        if location == 0:
            requests = np.random.poisson(2)

        if location == 1:
            requests = np.random.poisson(12)   #MDP Poisson distribution
        
        if location == 2:
            requests = np.random.poisson(4)    #MDP Poisson distribution
            
        if location == 3:
            requests = np.random.poisson(7)    #MDP Poisson distribution

        if location == 4:
            requests = np.random.poisson(8)    #MDP Poisson distribution  
            
        if requests > 15:
            requests = 15

        possible_actions_index = random.sample(range(0, (m-1)*m), requests) # (0,0) is not considered as customer request
        possible_actions_index.append(20) #add the index of No-OP action (0, 0)
        actions = [self.action_space[i] for i in possible_actions_index]

        print('Number of actions available', len(actions))
        return possible_actions_index, actions   



    def reward_func(self, state, action, Time_matrix):
        """Takes in state, action and Time-matrix and returns the reward"""
        if action[0] == action[1]:
            reward = -C 
            return reward

        #print('reward:' ,state, action)
        p = action[0]
        q = action[1]
        i = state[0]
        time = state[1]
        day = state[2]
        #print('reward vals:', (p, q, i, time, day))
        t_pq = Time_matrix[p][q][time][day]
        t_ip = Time_matrix[i][p][time][day]
        
        
        reward = (R*t_pq)-(C*(t_pq+t_ip))
        return reward


    def next_state_func(self, state, action, Time_matrix):
        """Takes state and action as input and returns next state"""
        
        #print('next_state :', state, action)
        p = action[0]
        q = action[1]
        i = state[0]
        time_curr = state[1]
        day_curr = state[2]
        #print('next_state_vals :', (p, q, i, time_curr, day_curr))
        time_next = time_curr + Time_matrix[p][q][time_curr][day_curr]

        day_next = int((day_curr+int(time_next/24)) % 7)
        time_next = int(time_next % 24)
            
        next_state = (q,time_next,day_next)
        return next_state


    def reset(self):
        return self.action_space, self.state_space, self.state_init


#### Defining Time Matrix

In [3]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

#### Tracking the state-action pairs for checking convergence


In [4]:
#Defining a function to save the Q-dictionary as a pickle file
def save_pickle(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size, discount_factor=0.95, learning_rate=0.01,
                       epsilon=0.99, epsilon_decay=0.99, epsilon_min=0.01):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate        
        self.epsilon_max = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.model_history = None
        
        self.batch_size = 32
        #self.batch_size = 1
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()
    

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        # Write your code here: Add layers to your neural nets       

        # hidden layers
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))

        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))     
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model


    def get_action(self, cstate, all_actions, pos_act_ind):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment
        actions = all_actions[pos_act_ind]
        q_value = 0
        if np.random.rand() <= self.epsilon_max:
            # explore: choose a random action from all possible actions
            print('Exploring')
            action = random.choice(actions)
        else:
            # choose the action with the highest q(s, a)
            # the first index corresponds to the batch size, so
            # reshape state to (1, state_size) so that the first index corresponds to the batch size
            print('Exploiting')
            #cstate = cstate.reshape(1, self.state_size) 
            q_value = self.model.predict(x=cstate)
            max_index = np.argmax(q_value[0])
            action = all_actions[max_index] if max_index in pos_act_ind else random.choice(actions)
        print('Selected action ', action)    
        return action, q_value
        

    def append_sample(self, state, action, reward, next_state, done):
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, done))
    
    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_input = np.zeros((self.batch_size, self.state_size))
            update_output = np.zeros((self.batch_size, self.state_size))
            
            actions, rewards, done = [], [], []
            
            for i in range(self.batch_size):
                state, action, reward, next_state, done_boolean = mini_batch[i]
                
                # Write your code from here
                # 1. Predict the target from earlier model
                update_input[i] = env.state_encod_arch1(state)
                actions.append(action)
                rewards.append(reward)
                update_output[i] = env.state_encod_arch1(next_state)
                done.append(done_boolean)
                
            # 2. Get the target for the Q-network
            
            target = self.model.predict(update_input)
            target_qval = self.model.predict(update_output)
            #print(target, target.shape)
            #print(target.shape, target_qval.shape)

            #3. Update your 'update_output' and 'update_input' batch
            for i in range(self.batch_size):
                #print(i, actions[i])
                if done[i]:
                    target[i][actions[i]] = rewards[i]
                    #target[i] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])
                    #target[i] = rewards[i] + self.discount_factor * np.max(target_qval[i])
                
        # 4. Fit your model and track the loss values
            return self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=1)
            
            
    def save(self, name):
        self.model.save(name)

In [ ]:
# to store rewards in each episode
rewards_per_episode, episodes, q_vals_per_episode, loss = [], [], [], []

# make dir to store model weights
if not os.path.exists("saved_model_weights"):
    os.mkdir("saved_model_weights")

# n_episodes
n_episodes = 150

### DQN block

In [ ]:
 # Call all the initialised variables of the environment
env = CabDriver()
#Call the DQN agent
dqn = DQNAgent(env.state_size, env.action_size)

for episode in range(n_episodes):

    # Write code here
    # Call the environment
   
    _,_,curr_state = env.reset()
    state_size = env.state_size
    pos_act_ind, actions = env.requests(curr_state)
    action = random.choice(actions)
    #action_size = len(actions)
    reward = 0
    curr_time = 0
    q_val_list = []
    #print(curr_state)
    
    
    terminal_state = False
    print("Episode :", episode)
    
    while not terminal_state:
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        encoded_state = env.state_encod_arch1(curr_state)
        #encoded_state = np.reshape(encoded_state, [1, env.state_size])
        action, q_value = dqn.get_action(encoded_state, env.action_space, pos_act_ind)
        # 2. Evaluate your reward and next state
        reward = reward + env.reward_func(curr_state, action, Time_matrix)
        next_state = env.next_state_func(curr_state,action,Time_matrix)
        
        q_val_list.append(q_value)
        
        p = action[0]
        q = action[1]
        i = curr_state[0]
        time = curr_state[1]
        day = curr_state[2]
        print(action)
        curr_time = curr_time + Time_matrix[i][p][time][day]
        
        
        day = int((day+int(time/24)) % 7)
        time = int(time % 24)
        
        curr_time = curr_time + Time_matrix[p][q][time][day]
        day = int((day+int(time/24)) % 7)
        time = int(time % 24)
        # 3. Append the experience to the memory
        dqn.append_sample(curr_state, action, reward, next_state, terminal_state)
        curr_state = next_state
        
        # 4. Train the model by calling function agent.train_model
        history = dqn.train_model()
        # 5. Keep a track of rewards, Q-values, loss
        
        if history:
            print('rewards: ', reward, 'q-value: ', q_value)
            print('loss:', history.history['loss'])
        
        if curr_time >= 24*30:
            terminal_state = True
    
        # store total reward obtained in this episode
    rewards_per_episode.append(reward)
    episodes.append(episode)
        
    if dqn.epsilon_max > dqn.epsilon_min:
        dqn.epsilon_max *= dqn.epsilon_decay        
        
        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                26912     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 21)                693       
Total params: 28,661
Trainable params: 28,661
Non-trainable params: 0
_________________________________________________________________
Number of actions available 13
Episode : 0
Exploring
Selected action  [4 0]
[4 0]
Exploring
Selected action  [2 3]
[2 3]
Exploring
Selected action  [4 0]
[4 0]
Exploiting
Selected action  [1 0]
[1 0]
Exploring
Selected action  [0 0]
[0 0]
Exploring
Selected action  [4 0]
[4 0]
Exploring
Selected action  [4 0]
[4 0]
Exploring
Selected action  [2 3]
[2 3]
Exploring
Selected action  [0 1]
[0 1]
Explori

rewards:  51.0 q-value:  0
loss: [617.7643432617188]
Exploring
Selected action  [0 4]
[0 4]
Epoch 1/1
32/32 [==============================] - 0s 164us/step - loss: 510.9820
rewards:  35.0 q-value:  0
loss: [510.9819641113281]
Exploring
Selected action  [4 1]
[4 1]
Epoch 1/1
32/32 [==============================] - 0s 118us/step - loss: 546.4935
rewards:  39.0 q-value:  0
loss: [546.4935302734375]
Exploring
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 165us/step - loss: 464.7519
rewards:  22.0 q-value:  0
loss: [464.7519226074219]
Exploring
Selected action  [1 3]
[1 3]
Epoch 1/1
32/32 [==============================] - 0s 164us/step - loss: 630.7762
rewards:  13.0 q-value:  0
loss: [630.7761840820312]
Exploring
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 194us/step - loss: 540.7274
rewards:  -11.0 q-value:  0
loss: [540.7274169921875]
Exploring
Selected action  [0 1]
[0 1]
Epoch 1/1
32/32 [====================

rewards:  7.0 q-value:  0
loss: [577.791748046875]
Exploring
Selected action  [2 3]
[2 3]
Epoch 1/1
32/32 [==============================] - 0s 166us/step - loss: 359.1054
rewards:  13.0 q-value:  0
loss: [359.10540771484375]
Exploring
Selected action  [3 2]
[3 2]
Epoch 1/1
32/32 [==============================] - 0s 222us/step - loss: 367.2470
rewards:  25.0 q-value:  0
loss: [367.2469787597656]
Exploring
Selected action  [2 3]
[2 3]
Epoch 1/1
32/32 [==============================] - 0s 175us/step - loss: 307.2423
rewards:  37.0 q-value:  0
loss: [307.2422790527344]
Exploring
Selected action  [1 0]
[1 0]
Epoch 1/1
32/32 [==============================] - 0s 170us/step - loss: 305.5494
rewards:  63.0 q-value:  0
loss: [305.54937744140625]
Exploring
Selected action  [4 2]
[4 2]
Epoch 1/1
32/32 [==============================] - 0s 153us/step - loss: 426.3515
rewards:  36.0 q-value:  0
loss: [426.3515319824219]
Exploring
Selected action  [1 0]
[1 0]
Epoch 1/1
32/32 [=====================

rewards:  -80.0 q-value:  0
loss: [251.00128173828125]
Exploring
Selected action  [2 3]
[2 3]
Epoch 1/1
32/32 [==============================] - 0s 177us/step - loss: 409.0840
rewards:  -56.0 q-value:  0
loss: [409.08404541015625]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 106us/step - loss: 329.4883
rewards:  -61.0 q-value:  0
loss: [329.48828125]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 135us/step - loss: 379.3337
rewards:  -62.0 q-value:  0
loss: [379.33367919921875]
Exploring
Selected action  [2 3]
[2 3]
Epoch 1/1
32/32 [==============================] - 0s 133us/step - loss: 399.9736
rewards:  -55.0 q-value:  0
loss: [399.9736328125]
Exploring
Selected action  [1 0]
[1 0]
Epoch 1/1
32/32 [==============================] - 0s 138us/step - loss: 396.6104
rewards:  -37.0 q-value:  0
loss: [396.61041259765625]
Exploring
Selected action  [4 2]
[4 2]
Epoch 1/1
32/32 [===================

Epoch 1/1
32/32 [==============================] - 0s 93us/step - loss: 426.0220
rewards:  -70.0 q-value:  0
loss: [426.0220031738281]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 182us/step - loss: 352.2650
rewards:  -84.0 q-value:  0
loss: [352.2650146484375]
Exploring
Selected action  [3 0]
[3 0]
Epoch 1/1
32/32 [==============================] - 0s 179us/step - loss: 583.6471
rewards:  -107.0 q-value:  0
loss: [583.6470947265625]
Exploring
Selected action  [3 2]
[3 2]
Epoch 1/1
32/32 [==============================] - 0s 112us/step - loss: 527.8972
rewards:  -75.0 q-value:  0
loss: [527.897216796875]
Exploring
Selected action  [3 0]
[3 0]
Epoch 1/1
32/32 [==============================] - 0s 135us/step - loss: 514.6439
rewards:  -73.0 q-value:  0
loss: [514.6438598632812]
Exploring
Selected action  [1 0]
[1 0]
Epoch 1/1
32/32 [==============================] - 0s 154us/step - loss: 428.7250
rewards:  -75.0 q-value:  0
loss: [428.72497

...

rewards:  -42.0 q-value:  [[0.         0.         0.         0.         0.         0.17813192
  0.         0.         0.         0.         0.         0.
  0.0278997  0.         0.         0.01525487 0.         0.11890816
  0.         0.02513181 0.        ]]
loss: [5239.642578125]
Exploring
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 207us/step - loss: 1915.7162
rewards:  -47.0 q-value:  0
loss: [1915.7161865234375]
Exploring
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 199us/step - loss: 7636.0635
rewards:  -52.0 q-value:  0
loss: [7636.0634765625]
Exploiting
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 248us/step - loss: 1891.6599
rewards:  -57.0 q-value:  [[0.         0.         0.         0.         0.         0.19964492
  0.         0.         0.         0.02480842 0.         0.
  0.00202443 0.         0.         0.04684129 0.         0.06943158
  0.         0.0444722

Epoch 1/1
32/32 [==============================] - 0s 160us/step - loss: 12611.2725
rewards:  -210.0 q-value:  0
loss: [12611.2724609375]
Exploring
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 157us/step - loss: 9326.6885
rewards:  -214.0 q-value:  0
loss: [9326.6884765625]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 174us/step - loss: 6908.3311
rewards:  -234.0 q-value:  0
loss: [6908.3310546875]
Exploring
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 120us/step - loss: 1026.3635
rewards:  -239.0 q-value:  0
loss: [1026.363525390625]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 130us/step - loss: 12550.1240
rewards:  -259.0 q-value:  0
loss: [12550.1240234375]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 108us/step - loss: 14257.0068
rewards:  -259.0 q-value:  0
loss: 

rewards:  -492.0 q-value:  0
loss: [3106.478515625]
Exploring
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 127us/step - loss: 6620.6484
rewards:  -498.0 q-value:  0
loss: [6620.6484375]
Exploring
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 143us/step - loss: 3329.0947
rewards:  -503.0 q-value:  0
loss: [3329.0947265625]
Exploiting
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 106us/step - loss: 4010.6382
rewards:  -509.0 q-value:  [[0.         0.         0.         0.         0.         0.34437835
  0.         0.         0.         0.         0.         0.
  0.03924166 0.         0.         0.         0.0516016  0.11003996
  0.         0.03479169 0.        ]]
loss: [4010.63818359375]
Exploiting
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==============================] - 0s 175us/step - loss: 5476.3652
rewards:  -515.0 q-value:  [[0.         0.         0.         0.      

rewards:  -659.0 q-value:  0
loss: [3184.498291015625]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 124us/step - loss: 11315.5166
rewards:  -670.0 q-value:  0
loss: [11315.5166015625]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 86us/step - loss: 2319.9194
rewards:  -671.0 q-value:  0
loss: [2319.91943359375]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 182us/step - loss: 11135.9004
rewards:  -679.0 q-value:  0
loss: [11135.900390625]
Exploring
Selected action  [0 0]
[0 0]
Epoch 1/1
32/32 [==============================] - 0s 150us/step - loss: 6374.4678
rewards:  -684.0 q-value:  0
loss: [6374.4677734375]
Exploring
Selected action  [2 4]
[2 4]
Epoch 1/1
32/32 [==============================] - 0s 111us/step - loss: 3132.1968
rewards:  -691.0 q-value:  0
loss: [3132.19677734375]
Exploring
Selected action  [2 0]
[2 0]
Epoch 1/1
32/32 [==========

Epoch 1/1
32/32 [==============================] - 0s 180us/step - loss: 3469.4907
rewards:  -11.0 q-value:  0
loss: [3469.49072265625]
Exploring
Selected action  [4 0]
[4 0]
Epoch 1/1


In [ ]:
# make directory
if not os.path.exists("saved_pickle_files"):
    os.mkdir("saved_pickle_files")

# save rewards_per_episode
save_pickle(rewards_per_episode, "saved_pickle_files/rewards_per_episode")


# plot results
with open('saved_pickle_files/rewards_per_episode.pkl', 'rb') as f:
    rewards_per_episode = pickle.load(f)

plt.plot(list(range(len(rewards_per_episode))), rewards_per_episode)
plt.xlabel("episode number")
plt.ylabel("reward per episode")

# save plots in saved_plots/ directory
plt.savefig('rewards.png')

print("Average reward of last 100 episodes is {0}".format(np.mean(rewards_per_episode[-100:])))

### Tracking Convergence

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()